In [1]:
#!pip install pyarrow
#!pip install tensorflow

In [1]:
import numpy as np
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset
import pandas as pd
import pyarrow
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
#CONSTANTS
exercises_dict = {
    'Abduction': 0,
    'Bird': 1,
    'Bridge': 2,
    'Knee': 3,
    'Shoulder': 4,
    'Squat' : 5,
    'Stretch' : 6
}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/prep_dataset.parquet', engine='pyarrow')


In [5]:
#indices = df.index
#labels = df.columns

X = df.drop(['Participant', 'Exercise', 'Set', 'Camera'], axis=1)
y_exercise = df['Exercise']
#y_set = df['Set']
del df
gc.collect()

0

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y_exercise, test_size=0.2, shuffle = False, random_state=42)
del X
del y_exercise
gc.collect()
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(np.array([exercises_dict[y] for y in y_train.values]), dtype=torch.int64)

X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(np.array([exercises_dict[y] for y in y_test.values]), dtype=torch.int64)

In [7]:
get_mlp = lambda: torch.nn.Sequential(

    torch.nn.Flatten(),
    torch.nn.Linear(100, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 7),
)

get_cnn = lambda: torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Conv1d(1, out_channels=32, kernel_size=3, stride=1, padding=1),
    torch.nn.ReLU(),
    torch.nn.MaxPool1d(kernel_size=2, stride=2),
    torch.nn.Linear(32 * 50, 7),
)

In [8]:

"""
Return a modified dataset cls that can insert MNIST like images into larger
frames with an option for random shifts, scrambling the images in a
consistent way (using the same shuffling for all images) and adding random
Gaussian noise (to the base data, noise is always the same for a given
image).
"""

class CustomDataset(Dataset):
    def __init__(self, x_train, y_train, transform=None):
        self.x_train = x_train
        self.y_train = y_train
        self.transform = transform

    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        sample = {'input': self.x_train[idx], 'label': self.y_train[idx]}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [9]:
learning_rate = 1e-3
num_epochs = 10

# Create an instance of the custom dataset
custom_dataset = CustomDataset(X_train, y_train)

# Create a DataLoader
batch_size = 64
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

model = get_mlp()
model.train()
#model_path = 'MLP.path'
#model.load_state_dict(torch.load(model_path))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
#def train_epoch()
for epoch in range(num_epochs):
    losses = []
    for batch in custom_dataloader:
        inputs, labels = batch['input'], batch['label']
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.mean(losses):.4f}')

In [ ]:
torch.save(model.state_dict(),'MLP.path')

In [ ]:
model.eval()

# Make predictions on the test set
with torch.no_grad():
    predictions = model(X_test)

# Convert predictions to class labels
predicted_labels = torch.argmax(predictions, dim=1)

# Convert tensors to numpy arrays
y_test_np = y_test.numpy()
predicted_labels_np = predicted_labels.numpy()

# Print accuracy
accuracy = accuracy_score(y_test_np, predicted_labels_np)
print(f'Accuracy: {accuracy:.4f}')

# Print classification report
print('Classification Report:')
print(classification_report(y_test_np, predicted_labels_np, labels=range(7)))

In [12]:
learning_rate = 1e-3
num_epochs = 1


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Create an instance of the custom dataset
#X_train, y_train = X_train.to(device), y_train.to(device)

custom_dataset = TensorDataset(X_train, y_train)

# Create a DataLoader
batch_size = 64
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

model = get_cnn()
model.train()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
for epoch in range(num_epochs):
    losses = []
    for batch in custom_dataloader:
        inputs, labels = batch['input'], batch['label']
        print(inputs.shape)
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.mean(losses):.4f}')

TypeError: ignored